<a href="https://colab.research.google.com/github/duridudu/dupository/blob/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0%20chapter%2005_3_%ED%8A%B8%EB%A6%AC%EC%9D%98%20%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **05-3 트리의 앙상블**

* 정형 데이터의 끝판왕인 **앙상블** 학습 알고리즘 

 (비정형 데이터는 전통적인 머신러닝 방법으로 학습하기 어려워서 신경망 알고리즘을 많이 씀)

 </br>


## **랜덤 포레스트** 

* 앙상블 학습의 대표 주자. 결정 트리를 랜덤하게 만들어 결정트리의 숲을 만듦. 

  그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듦. 

  ==> **부트스트랩 샘플** 방식. 훈련 세트 수만큼 샘플 무작위로 뽑아서(나왔던 것 다시 넣고) 만든 샘플.

  by **RandomForestClassifier** : 전체 특성 개수의 제곱근만큼의 특성을 선택. 

    cf. 회귀모델인 RandomForestRegressor는 전체 특성을 사용함
</br>

* 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아줌.

 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음. 따라서 종종 기본 매개변수 설정만으로도 아주좋은 결과 냄.

</br>

* 랜덤 포레스트는 결정 트리의 앙상블이기 대문에 DecisionTreeClassifier가 제공하는 주요 매개변수를 모두 제공함.

 ==> criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등

</br>

* **OOB(Out Of Bound) 샘플** 

  : 부트스트랩에서 뽑히지 않은 샘플. RandomForestClassifier에는 이를 가지고 자체적으로 모델을 평가할 수 있음. (마치 검증 세트의 역할)

  by. oob_score 매개변수 True 지정

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine=pd.read_csv('https://bit.ly/wine_csv_data')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)


In [2]:
#교차 검증. n_jobs 매개변수를 -1로 설정하여 최대한 쓸 수 있는 코어로 훈련
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

#리턴 트레인 스코어 True로 설정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환함. 
#(훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는데 용이함.)
scores=cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
#각 결정 트리의 특성 중요도를 취합한 특성 중요도
rf.fit(train_input, train_target)
print(rf.feature_importances_)
#1절 결정 트리에서 만든 특성 중요도는 2번째 당도가 독보적으로 높았는데, 
#랜덤 포레스트에서는 좀 더 고루고루 높아짐. 
#==>하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음. 
#==>과대적합을 줄이고 일반화 성능을 높이는 데 도움을 줌.

[0.23167441 0.50039841 0.26792718]


In [4]:
#OOB 샘플
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)
#교차 검증에서 얻은 점수와 매우 비슷한 결

0.8934000384837406


## **엑스트라 트리**
* 기본으로 100개의 결정 트리 훈련. 그러나 부트스트랩 샘플을 사용하지 않고 전체 훈련 세트를 사용 

 대신, 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함. 

 ==> 성능이 낮아지겠지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있음. 

 by. **ExtraTreesClassifier**

* 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 함. 

* 하지만 노드를 랜덤으로 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점. (최적의 분할 찾느라 시간 안써도 돼서)

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
#특성 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)
#역시 두번째 sugar의 중요도가 가장 높음

[0.20183568 0.52242907 0.27573525]


## **그레이디언트 부스팅**
 * 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방식

 * **GradientBoostingClassifier** : 기본적으로 깊이가 3인 결정 트리를 100개 사용함. 

 * 깊이가 얕기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음.

 * 경사 하강법을 사용하여 트리를 앙상블에 추가함. 분류에서는 로지스틱 손실 함수 사용, 회귀에서는 평균제곱오차 함수 사용

* 일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있지만, 순서대로 트리를 추가하기 때문에 훈련속도가 느림. (GradientBoostingClassifier에는 n_jobs 매개변수가 없음)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#과대적합이 거의 되지 않음!

0.8881086892152563 0.8720430147331015


In [9]:
gb=GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                              random_state=42)
scores=cross_validate(gb, train_input, train_target,
                      return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#결정 트리 개수 500개로 5배나 늘렸지만 과대적합을 잘 억제하고 있음

0.9464595437171814 0.8780082549788999


In [10]:
#중요도 제공
gb.fit(train_input, train_target)
print(gb.feature_importances_)

#그레디언트 부스팅이 랜덤 포레스트보다 일부 특성(sugar)에 더 집중함

[0.15872278 0.68010884 0.16116839]


## **히스토그램 기반 그레이디언트 부스팅**
* 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘

* 입력 특성을 256개의 구간으로 나누고 하나를 떼어놓고(노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음)

  256개의 구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용. (입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음)

* by. **HistGradientBoositngClassifier** 

 ==> 트리 개수 지정에 n_estimators 대신 max_iter 사용

* 특성 중요도 계산 by. **permutation_importance()**

In [11]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능 제공

0.9321723946453317 0.8801241948619236


In [14]:
#특성 중요도 계산
#n_repeats은 랜덤하게 섞을 횟수 지정
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result=permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [15]:
#특성 중요도 in 테스트 세트
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

#그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중
#이런 분석을 통해 모델을 실전에 투입했을 때 어떤 특성에 관심을 둘지 예상할 수 있음

[0.05969231 0.20238462 0.049     ]


In [16]:
#테스트 세트에서의 성능 최종적으로 확인
hgb.score(test_input, test_target)

0.8723076923076923

## **다른 부스팅 방법들**
* **XGBoost** : from xgboost import XGBClassifier

* **LightGBM** : from lightgmb import LGBMClassifier